# Генерация звука с помощью аддитивного синтеза в Python

## Введение в аддитивный синтез

Аддитивный синтез - это метод генерации звука путем сложения простых звуковых волн (обычно синусоидальных) с различными частотами, амплитудами и фазами. Этот подход основан на теореме Фурье, которая утверждает, что любой сложный звуковой сигнал может быть представлен как сумма простых синусоидальных волн.

## Основные понятия

1. **Гармоники** - синусоидальные компоненты с частотами, кратными основной частоте (фундаментальной)

2. **Амплитуда** - громкость каждой гармоники

3. **Фаза** - временное смещение гармоники

4. **Огибающая** - изменение амплитуды звука во времени

## Математическая основа

Звуковой сигнал можно представить как:

y(t) = Σ Aₙ * sin(2π * n * f₀ * t + φₙ)

где:

Aₙ - амплитуда n-ой гармоники

f₀ - основная частота

n - номер гармоники (1 - основная частота, 2 - первая гармоника и т.д.)

φₙ - фаза n-ой гармоники

t - время

## Практическая реализация в Python

1.  Установка необходимых библиотек

In [ ]:
!sudo apt-get install libportaudio2
!pip install sounddevice

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio

2. Генерация простого синусоидального тона

In [ ]:
# Параметры сигнала
sample_rate = 44100  # частота дискретизации (Гц)
duration = 2.0       # длительность (сек)
frequency = 440.0    # частота (Гц) - нота Ля первой октавы
amplitude = 0.5      # амплитуда (0-1)

# Генерация временной оси
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)

# Генерация синусоидального сигнала
signal = amplitude * np.sin(2 * np.pi * frequency * t)

# Визуализация первых 1000 отсчетов
plt.plot(t[:1000], signal[:1000])
plt.xlabel('Время (с)')
plt.ylabel('Амплитуда')
plt.title('Синусоидальный сигнал 440 Гц')
plt.show()

# Воспроизведение звука
Audio(signal, rate=sample_rate)

3. Аддитивный синтез с несколькими гармониками

In [ ]:
def additive_synthesis(fundamental_freq, harmonics, amplitudes, duration, sample_rate=44100):
    """
    Генерация звука с помощью аддитивного синтеза
    
    Параметры:
        fundamental_freq: основная частота (Гц)
        harmonics: список гармоник (например, [1, 2, 3] для 1-й, 2-й, 3-й гармоник)
        amplitudes: список амплитуд для каждой гармоники
        duration: длительность звука (сек)
        sample_rate: частота дискретизации (Гц)
    """
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    signal = np.zeros_like(t)
    
    for harmonic, amplitude in zip(harmonics, amplitudes):
        freq = fundamental_freq * harmonic
        signal += amplitude * np.sin(2 * np.pi * freq * t)
    
    # Нормализация сигнала
    signal /= np.max(np.abs(signal))
    
    return t, signal

# Пример: создаем звук с основной частотой 220 Гц и 5 гармониками
fundamental = 220
harmonics = [1, 2, 3, 4, 5]
amplitudes = [1.0, 0.5, 0.3, 0.2, 0.1]

t, signal = additive_synthesis(fundamental, harmonics, amplitudes, 2.0)

# Воспроизведение звука
Audio(signal, rate=sample_rate)

4. Визуализация спектра

In [ ]:
from scipy.fft import fft, fftfreq

def plot_spectrum(signal, sample_rate):
    n = len(signal)
    yf = fft(signal)
    xf = fftfreq(n, 1 / sample_rate)
    
    plt.figure(figsize=(10, 4))
    plt.plot(xf[:n//2], np.abs(yf[:n//2]))
    plt.xlabel('Частота (Гц)')
    plt.ylabel('Амплитуда')
    plt.title('Спектр сигнала')
    plt.grid()
    plt.show()

plot_spectrum(signal, sample_rate)

5. Создание более сложных тембров

Можно имитировать различные музыкальные инструменты, подбирая амплитуды гармоник:

In [ ]:
# Параметры для имитации флейты (более слабые высшие гармоники)
flute_harmonics = [1, 2, 3, 4, 5, 6]
flute_amplitudes = [1.0, 0.7, 0.3, 0.1, 0.05, 0.01]

# Параметры для имитации кларнета (нечетные гармоники сильнее)
clarinet_harmonics = [1, 3, 5, 7, 9]
clarinet_amplitudes = [1.0, 0.8, 0.6, 0.4, 0.2]

# Генерация и воспроизведение звука флейты
_, flute_sound = additive_synthesis(440, flute_harmonics, flute_amplitudes, 2.0)
Audio(flute_sound, rate=sample_rate)

In [ ]:
# Генерация и воспроизведение звука кларнета
_, clarinet_sound = additive_synthesis(440, clarinet_harmonics, clarinet_amplitudes, 2.0)
Audio(clarinet_sound, rate=sample_rate)

6. Добавление огибающей

Для более естественного звучания добавим ADSR-огибающую (Attack-Decay-Sustain-Release):

In [ ]:
def apply_adsr_envelope(signal, sample_rate, attack, decay, sustain_level, release):
    """
    Применение ADSR-огибающей к сигналу
    
    Параметры:
        signal: входной сигнал
        sample_rate: частота дискретизации
        attack: время атаки (сек)
        decay: время спада (сек)
        sustain_level: уровень поддержки (0-1)
        release: время затухания (сек)
    """
    n_samples = len(signal)
    envelope = np.zeros(n_samples)
    
    # Расчет количества отсчетов для каждой фазы
    attack_samples = int(attack * sample_rate)
    decay_samples = int(decay * sample_rate)
    release_samples = int(release * sample_rate)
    sustain_samples = n_samples - attack_samples - decay_samples - release_samples
    
    if sustain_samples < 0:
        raise ValueError("Общее время фаз превышает длительность сигнала")
    
    # Фаза атаки (линейный рост от 0 до 1)
    envelope[:attack_samples] = np.linspace(0, 1, attack_samples)
    
    # Фаза спада (линейный спад от 1 до sustain_level)
    envelope[attack_samples:attack_samples+decay_samples] = np.linspace(1, sustain_level, decay_samples)
    
    # Фаза поддержки (постоянный уровень)
    envelope[attack_samples+decay_samples:attack_samples+decay_samples+sustain_samples] = sustain_level
    
    # Фаза затухания (линейный спад от sustain_level до 0)
    envelope[attack_samples+decay_samples+sustain_samples:] = np.linspace(sustain_level, 0, release_samples)
    
    return signal * envelope

# Генерация сигнала с огибающей
_, signal = additive_synthesis(440, [1, 2, 3], [1, 0.5, 0.2], 3.0)
signal_with_env = apply_adsr_envelope(signal, sample_rate, 
                                    attack=0.1, decay=0.2, 
                                    sustain_level=0.7, release=0.5)

# Визуализация огибающей
plt.plot(np.linspace(0, 3.0, len(signal_with_env)), signal_with_env)
plt.title('Сигнал с ADSR-огибающей')
plt.xlabel('Время (с)')
plt.ylabel('Амплитуда')
plt.show()

Audio(signal_with_env, rate=sample_rate)

## Генерация шумовых звуков с помощью аддитивного синтеза

### Теоретические основы шумов

Шумовые звуки принципиально отличаются от периодических тем, что не имеют четкой гармонической структуры. В музыке и звуковом дизайне используются несколько основных типов шумов:

**Белый шум** - содержит все частоты с равной интенсивностью

**Розовый шум** - интенсивность уменьшается на 3 дБ на октаву (более "теплый" звук)

**Красный (Броуновский) шум** - интенсивность уменьшается на 6 дБ на октаву

**Голубой шум** - интенсивность увеличивается на 3 дБ на октаву

**Фиолетовый шум** - интенсивность увеличивается на 6 дБ на октаву

### Генерация белого шума
Самый простой в реализации - белый шум, который можно создать с помощью случайных чисел:

In [ ]:
def generate_white_noise(duration, sample_rate=44100, amplitude=0.5):
    """Генерация белого шума"""
    num_samples = int(sample_rate * duration)
    noise = amplitude * np.random.uniform(-1, 1, num_samples)
    return noise

# Генерация и воспроизведение
white_noise = generate_white_noise(2.0)


# Визуализация спектра
plot_spectrum(white_noise[:10000], sample_rate)  # берем первые 10k отсчетов для наглядности

Audio(white_noise, rate=sample_rate)

## Генерация розового шума

Розовый шум требует более сложного подхода. Один из методов - фильтрация белого шума:

In [ ]:
def generate_pink_noise(duration, sample_rate=44100, amplitude=0.5):
    """Генерация розового шума с использованием фильтрации"""
    num_samples = int(sample_rate * duration)
    white = np.random.randn(num_samples)
    
    # Применяем фильтр для получения розового шума
    b = [0.049922035, -0.095993537, 0.050612699, -0.004408786]
    a = [1, -2.494956002, 2.017265875, -0.522189400]
    pink = np.zeros_like(white)
    
    # Реализация фильтра
    for i in range(len(white)):
        pink[i] = (white[i] * b[0] + 
                  (white[i-1] * b[1] if i > 0 else 0) + 
                  (white[i-2] * b[2] if i > 1 else 0) + 
                  (white[i-3] * b[3] if i > 2 else 0) - 
                  (pink[i-1] * a[1] if i > 0 else 0) - 
                  (pink[i-2] * a[2] if i > 1 else 0) - 
                  (pink[i-3] * a[3] if i > 2 else 0))
    
    # Нормализация
    pink /= np.max(np.abs(pink)) * amplitude
    return pink

# Генерация и воспроизведение
pink_noise = generate_pink_noise(2.0)

plot_spectrum(pink_noise[:10000], sample_rate)

Audio(pink_noise, rate=sample_rate)

## Аддитивный синтез шумовых компонентов

Мы можем комбинировать гармонические и шумовые компоненты для создания более сложных звуков:

In [ ]:
def hybrid_sound(fundamental_freq, harmonics, amplitudes, 
                noise_type='white', noise_ratio=0.3, duration=2.0):
    """Создание гибридного звука с гармониками и шумом"""
    # Генерация гармонической части
    t, harmonic_part = additive_synthesis(fundamental_freq, harmonics, amplitudes, duration)
    
    # Генерация шумовой части
    if noise_type == 'white':
        noise_part = generate_white_noise(duration, amplitude=1.0)
    elif noise_type == 'pink':
        noise_part = generate_pink_noise(duration, amplitude=1.0)
    else:
        raise ValueError("Unknown noise type")
    
    # Смешивание с учетом соотношения
    mixed = (1 - noise_ratio) * harmonic_part + noise_ratio * noise_part
    mixed /= np.max(np.abs(mixed))  # Нормализация
    
    return t, mixed

# Пример: звук флейты с белым шумом (имитация дыхания)
t, flute_with_breath = hybrid_sound(440, [1, 2, 3, 4], [1, 0.6, 0.3, 0.1], 
                                  noise_type='white', noise_ratio=0.2)
Audio(flute_with_breath, rate=sample_rate)



In [ ]:
# Пример: барабанный звук с розовым шумом
t, drum_sound = hybrid_sound(100, [1, 1.5, 2.3], [1, 0.4, 0.2],  # негармонические соотношения
                           noise_type='pink', noise_ratio=0.5, duration=1.0)
drum_sound = apply_adsr_envelope(drum_sound, sample_rate, 
                                attack=0.01, decay=0.1, 
                                sustain_level=0.0, release=0.0)
Audio(drum_sound, rate=sample_rate)

## Спектральный подход к генерации шума

Более точный контроль над шумом можно получить, генерируя его в частотной области:

In [ ]:
def spectral_noise(noise_profile, duration, sample_rate=44100):
    """
    Генерация шума с заданным спектральным профилем
    
    noise_profile: функция, возвращающая амплитуду для заданной частоты
    """
    num_samples = int(sample_rate * duration)
    freqs = np.fft.fftfreq(num_samples, d=1/sample_rate)
    
    # Генерация случайных фаз
    phases = 2 * np.pi * np.random.rand(num_samples // 2 + 1)
    
    # Создание спектра с заданным профилем
    spectrum = np.zeros(num_samples, dtype=complex)
    for i in range(1, num_samples // 2):
        freq = abs(freqs[i])
        amplitude = noise_profile(freq)
        spectrum[i] = amplitude * np.exp(1j * phases[i])
        spectrum[-i] = amplitude * np.exp(-1j * phases[i])  # зеркальная часть
    
    # Обратное преобразование Фурье
    noise = np.fft.ifft(spectrum).real
    noise /= np.max(np.abs(noise))  # Нормализация
    
    return noise

# Примеры профилей шума
def white_profile(freq):
    return 1.0 if freq > 0 else 0

def pink_profile(freq):
    return 1.0 / np.sqrt(freq) if freq > 0 else 0

def brown_profile(freq):
    return 1.0 / freq if freq > 0 else 0

# Генерация красного (броуновского) шума
red_noise = spectral_noise(brown_profile, 2.0)

plot_spectrum(red_noise[:10000], sample_rate)
Audio(red_noise, rate=sample_rate)

## Практическое применение: синтез ударных

Комбинируя шумы с гармониками и применяя различные огибающие, можно синтезировать ударные звуки:

In [ ]:
def synthesize_kick(duration=0.5, fundamental=60, sample_rate=44100):
    """Синтез бас-бочки (kick drum)"""
    # Гармоническая часть с падающей частотой (pitch envelope)
    t = np.linspace(0, duration, int(sample_rate * duration))
    freq_env = fundamental * np.exp(-5 * t)  # экспоненциальное падение частоты
    harmonic = np.sin(2 * np.pi * np.cumsum(freq_env) / sample_rate)
    
    # Шумовая часть (щелчок в атаке)
    noise = generate_white_noise(duration, amplitude=1.0)
    noise_env = np.exp(-50 * t)  # быстрый спад шума
    
    # Смешивание
    kick = 0.7 * harmonic * np.exp(-10 * t) + 0.3 * noise * noise_env
    kick /= np.max(np.abs(kick))
    
    return t, kick

t, kick = synthesize_kick()
Audio(kick, rate=sample_rate)

In [ ]:
def synthesize_snare(duration=0.5, fundamental=180, sample_rate=44100):
    """Синтез малого барабана (snare drum)"""
    # Гармоническая часть (короткий тон)
    t = np.linspace(0, duration, int(sample_rate * duration))
    harmonic = 0.3 * np.sin(2 * np.pi * fundamental * t) * np.exp(-20 * t)
    
    # Шумовая часть (основной звук)
    noise = generate_pink_noise(duration, amplitude=1.0)
    noise_env = np.exp(-15 * t)  # огибающая шума
    
    # Смешивание
    snare = 0.3 * harmonic + 0.7 * noise * noise_env
    snare /= np.max(np.abs(snare))
    
    return t, snare

t, snare = synthesize_snare()
Audio(snare, rate=sample_rate)